In [2]:
import pandas as pd
import os

In [5]:
data = pd.read_csv(os.path.join('..','data','experiment_data.csv'))

In [8]:
data

,Timestamp,Please enter your email address,Do you have a degree in computer science?,Have you taken a course in Machine Learning?,Owen is the great grandfather of Bob because Bob is the son of Paul and Owen is the grandfather of Paul,Owen is the great grandfather of Bob because Owen is the great grandfather of Stella and Stella is the sister of Bob,"Owen is the great grandfather of Bob because Bob is the son of Paul, Paul is the son of Mary and Mary is the daughter of Owen",Owen is the great grandfather of Bob because Bob is the nephew of Ruth and Owen is the grandfather of Ruth,Optional comments,Joan is the aunt of Paul because Joan is the aunt of Mike and Mike is the is the brother of Paul,...,Mary is the parent of Paul because Ruth is the sister of Paul and Mary is the parent of Ruth,Mary is the parent of Paul because Milly is the aunt of Paul and Milly is the sister of Mary,Mary is the parent of Paul because Mike is the brother of Paul and Mike is the child of Mary,Mary is the parent of Paul because Theresa is the grandparent of Paul and Theresa is the parent of Mary,Mary is the parent of Paul because Ruth is the sister of Paul and Ruth is the child of Mary,Mary is the parent of Paul because Jack is the uncle of Paul and Jack is the brother of Mary,Mary is the parent of Paul because Paul is the child of Mary,Mary is the parent of Paul because James is the spouse of Mary and Paul is the child of James,Optional comments.8,Optional general comments on the survey
0,2021/06/02 9:48:23 AM GMT+2,NaN,NaN,NaN,ranked #1 (excellent explanation),ranked #3 (bad explanation),ranked #2,Not an explanation,NaN,ranked #2,...,ranked #4,ranked #5,ranked #4,ranked #7,ranked #8,ranked #5,ranked #1 (excellent explanation),ranked #2,NaN,NaN
